In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Constants
c = 3e8  # Speed of light in m/s

# Transmission model for thin film
def transmission_function(n_sample, n_sub, omega, d, c=3e8):
    n0 = n_sample[0] + 1j * n_sample[1]  # Complex refractive index of the sample
    numerator = (2 * n0 * (n_sub + 1) * np.exp(1j * omega * d * (n0 - 1) / c)* np.exp(-1j * omega * 0))
    denominator = ((n0 + n_sub) * (1 + n0) + (n0 - 1) * (n_sub - n0) * np.exp(2j * omega * d * n0 / c))
    T_model = numerator / denominator
    return T_model

# Error function to minimize the difference between experimental and modeled transmission
def error_function(n_sample, n_sub, omega, d, T_exp, c=3e8):
    T_model = transmission_function(n_sample, n_sub, omega, d, c)
    dp = np.log(np.abs(T_model)) - np.log(np.abs(T_exp))  # Magnitude difference
    dq = np.angle(T_model) - np.angle(T_exp)  # Phase difference
    delta = np.sum(dp**2 + dq**2)
    return delta

# Compute refractive index from transmission data for a single frequency
def compute_refractive_index(T_exp, omega, n_sub, d=110e-9, n_sample_guess=None):
    if n_sample_guess is None:
        n_sample_guess = np.array([3.2, 0.0002])  # Initial guess for n1 (real) and n2 (imaginary)

    # Perform optimization for a single frequency
    result = minimize(error_function, n_sample_guess, args=(n_sub, omega, d, T_exp), method='Nelder-Mead')
    return result.x  # Return the optimized refractive index for this frequency

# Load the transmission data
data = np.loadtxt("/content/Normalised_PbI2_without cavity", skiprows=0)  # Ensure the correct file path

# Extract frequency, amplitude, and phase
frequencies = data[:, 0]  # Frequency in THz
amplitude = data[:, 1]  # Amplitude
phase = data[:, 2]  # Phase in radians

# Convert amplitude and phase into complex transmission values
T_measured = amplitude * np.exp(1j * phase)

# Known values
n_sub = 1 + 1j * 0.0002  # Refractive index of the substrate (Silicon + 10nm STO)
d = 110e-9  # Sample thickness in meters

# Convert frequency to angular frequency (omega = 2 * pi * frequency)
omega = 2 * np.pi * frequencies * 1e12  # Convert from THz to Hz

n_sample_guess = np.array([3.2, 0.0002])  # Initial guess for n1 (real) and n2 (imaginary)

# List to store the refractive index for each frequency
n_sample = []

# Calculate the complex refractive index for each frequency using the previous value as the initial guess
for i, T in enumerate(T_measured):
    omega_i = omega[i]

    # Adjust the initial guess based on the previous iteration's result
    if i > 0:
        n_sample_guess = n_sample[-1]  # Use the result from the previous iteration

    # Compute the refractive index for the current frequency
    n_current = compute_refractive_index(T, omega_i, n_sub, d, n_sample_guess)

    # Store the result
    n_sample.append(n_current)

# Convert to NumPy array for plotting
n_sample = np.array(n_sample)

# Extract real and imaginary parts for plotting
n_real = n_sample[:, 0]
n_imag = n_sample[:, 1]

# Compute permittivity (ε = n²)
permittivity_real = n_real**2 - n_imag**2
permittivity_imag = 2 * n_real * n_imag

# Plot refractive index
plt.figure(figsize=(13, 5))
plt.subplot(1, 2, 1)
plt.plot(frequencies, n_real, label="Re(n)", color='b')
plt.plot(frequencies, n_imag, label="Im(n)", color='r')
plt.xlabel("Frequency (THz)",fontsize=15, fontweight='bold')
plt.ylabel("Refractive Index",fontsize=15, fontweight='bold')
plt.xticks(fontsize=15, fontweight='bold')
plt.yticks(fontsize=15, fontweight='bold')
plt.xlim(0.6,2.5)
plt.legend()
plt.grid(False)

# Plot permittivity
plt.subplot(1, 2, 2)
plt.plot(frequencies, permittivity_real, label="Re(ε)", color='b')
plt.plot(frequencies, permittivity_imag, label="Im(ε)", color='r')
plt.xlabel("Frequency (THz)",fontsize=15, fontweight='bold')
plt.ylabel("Permittivity",fontsize=15, fontweight='bold')
plt.xticks(fontsize=15, fontweight='bold')
plt.yticks(fontsize=15, fontweight='bold')
plt.xlim(0.6,2.5)
plt.legend()
plt.grid(False)


plt.tight_layout()
plt.show()


